In [ ]:
!nvidia-smi

Sat May 28 11:28:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 14.7 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
     |████████████████████████████████| 6.6 MB 31.6 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from transformers import BertModel, BertTokenizer, BertConfig, BertForSequenceClassification, TFBertModel
train_df = pd.read_csv('/content/re-designed_train_df.csv')
train_df.drop('Unnamed: 0', axis=1, inplace=True)
train_df.head()

,SID,lemmatized,negative - AMBIENCE#GENERAL,negative - DRINKS#PRICES,negative - DRINKS#QUALITY,negative - DRINKS#STYLE_OPTIONS,negative - FOOD#PRICES,negative - FOOD#QUALITY,negative - FOOD#STYLE_OPTIONS,negative - LOCATION#GENERAL,...,positive - DRINKS#QUALITY,positive - DRINKS#STYLE_OPTIONS,positive - FOOD#PRICES,positive - FOOD#QUALITY,positive - FOOD#STYLE_OPTIONS,positive - LOCATION#GENERAL,positive - RESTAURANT#GENERAL,positive - RESTAURANT#MISCELLANEOUS,positive - RESTAURANT#PRICES,positive - SERVICE#GENERAL
0,1000:0,manzara sahane evet servis rezalet,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000:1,soguk su istemek soguk yok butun sulari disari...,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1000:2,yemek iyi hos lezzet iyi heyecan veri bi taraf...,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1000:3,servis fiyat cok fazla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000:4,lacivert kalite dusukluguyle cok sasirtti,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
a = pd.Series([0.2, 0.1, 0.3], index=['A', 'B', 'C'])
b = pd.Series([0.2, 0.2, 0.5], index=['A', 'D', 'E'])

In [ ]:
!pip list

In [3]:
texts = list(train_df["lemmatized"])
label_names = train_df.drop(["SID", "lemmatized"], axis=1).columns
labels = train_df[label_names].values

train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.1, random_state=42
)

sample_idx = 24
print(f'Sample: "{train_texts[sample_idx]}"')
print(f"Labels: {pd.Series(train_labels[sample_idx], label_names).to_dict()}")

Sample: "oturup saatlerce kalite caymak esliginde muhabbet etmek"
Labels: {'negative - AMBIENCE#GENERAL': 0, 'negative - DRINKS#PRICES': 0, 'negative - DRINKS#QUALITY': 0, 'negative - DRINKS#STYLE_OPTIONS': 0, 'negative - FOOD#PRICES': 0, 'negative - FOOD#QUALITY': 0, 'negative - FOOD#STYLE_OPTIONS': 0, 'negative - LOCATION#GENERAL': 0, 'negative - RESTAURANT#GENERAL': 0, 'negative - RESTAURANT#PRICES': 0, 'negative - SERVICE#GENERAL': 0, 'neutral - AMBIENCE#GENERAL': 0, 'neutral - DRINKS#PRICES': 0, 'neutral - DRINKS#QUALITY': 0, 'neutral - FOOD#PRICES': 0, 'neutral - FOOD#QUALITY': 0, 'neutral - FOOD#STYLE_OPTIONS': 0, 'neutral - LOCATION#GENERAL': 0, 'neutral - RESTAURANT#GENERAL': 0, 'neutral - RESTAURANT#PRICES': 0, 'neutral - SERVICE#GENERAL': 0, 'positive - AMBIENCE#GENERAL': 1, 'positive - DRINKS#QUALITY': 1, 'positive - DRINKS#STYLE_OPTIONS': 0, 'positive - FOOD#PRICES': 0, 'positive - FOOD#QUALITY': 0, 'positive - FOOD#STYLE_OPTIONS': 0, 'positive - LOCATION#GENERAL': 0, 'pos

In [8]:
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [6]:
len(train_labels)

980

In [ ]:
for i, label in enumerate(label_names):
  print(i)
  print(label)

In [ ]:
test_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
type(test_texts)

list

In [ ]:
# from collections import Counter
# train_df.drop(["SID", "lemmatized"], axis=1)

plotly

In [ ]:
61*16

976

In [ ]:
print("Length of train texts : ", len(train_texts))
print("Length of train texts : ", len(test_texts))
print("Length of train texts : ", len(train_labels))
print("Length of train texts : ", len(test_labels))

Length of train texts :  980
Length of train texts :  109
Length of train texts :  980
Length of train texts :  109


### A minimalistic Exploratory Data Analysis

In [ ]:
# text_lengths = [len(t.split()) for t in train_texts]
# ax = sns.histplot(data=text_lengths, kde=True, stat="density")
# ax.set_title("Texts length distribution (number of words)");

In [9]:
test_texts

['fiyatlari tl arasi porsiyonlari buyutmek mumkun',
 'methetmek hakketmek',
 'büyük merak gelip yemek haz etmek başka yer olmak',
 'rezervasyon sirasinda ozel gun soylediginiz takdir sampanya ikramlari guzel jest',
 'manzara yemek servis olmamis',
 'buranin doneri demek efsane degil',
 'yenmek kötü kebaplari ukalâ garson lüks restoran ödeyecegini alkol yemek fiyatiyla karsilasirsiniz',
 'd somo izgara et quesadilla sipariz et',
 'yakı çeşit çay biri mutlaka seviceksiniz',
 'kalaba oturmak gürültü çekmek değmek',
 'sıra lezzet',
 'keyif almak yok yok çay para değmek demek övmek övmek bitirmek mizuno pudding beğenmek',
 'şimdilerde göz ortalama mekân',
 'yer bulabilirseniz',
 'normal mekân normal tat',
 'yemek eski güzel özellikle mantı başarı',
 'olmak big chefs zincir',
 'maalesef son senet kalite servis epey sıkıntı',
 'hazir ürün yöre üretim ürün kullaniyor oluslari hosuma gitmek',
 'zengin acik büfe menü',
 'döner sarmak',
 'i̇çerisi yemek kokmak çalışmak gerek negatif enerji varmak

### Fine-tuning BERT

In [ ]:
MODEL_NAME = 'dbmdz/bert-base-turkish-128k-cased'
MAX_LENGTH = 39  # We truncate anything after the 200-th word to speed up training

config = BertConfig.from_pretrained(MODEL_NAME)

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, 
                            max_length=MAX_LENGTH, return_tensors="tf")
test_encodings = tokenizer(test_texts, truncation=True, padding=True, 
                           max_length=MAX_LENGTH, return_tensors="tf")

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))

sample_text = "yemek iyi ama servis kötü"
tokenizer.decode(tokenizer(sample_text)["input_ids"])

Downloading:   0%|          | 0.00/386 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

'[CLS] yemek iyi ama servis kötü [SEP]'

In [ ]:
test_dataset

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(39,), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(39,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(39,), dtype=tf.int32, name=None)}, TensorSpec(shape=(32,), dtype=tf.int64, name=None))>

In [ ]:
for x, y in train_dataset.take(1):
    print(x)

{'input_ids': <tf.Tensor: shape=(39,), dtype=int32, numpy=
array([     2,   5735,   5928, 100582,   1054,   3272,   2351,  12812,
            3,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(39,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(39,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>}


In [ ]:
train_labels.shape[1]

32

In [ ]:
# def compute_metrics(p): # Multiclass için bu, multilabel çalışmaz.
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred)
#     precision = precision_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, 
#             "precision": precision, 
#             "recall": recall, 
#             "f1": f1}

In [ ]:
transformer_model = TFBertModel.from_pretrained(
    MODEL_NAME, 
    output_hidden_states=False
)

Downloading:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

Some layers from the model checkpoint at dbmdz/bert-base-turkish-128k-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
bert = transformer_model.layers[0]

# The input is a dictionary of word identifiers 
input_ids = Input(shape=(MAX_LENGTH,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}

# Here we select the representation of the first token ([CLS]) for classification
# (a.k.a. "pooled representation")
bert_model = bert(inputs)[-1]

# Add a dropout layer and the output layer
dropout = Dropout(0.2, name='pooled_output')(bert_model)

x = tf.keras.layers.Dense(128, activation = tf.nn.silu,
                          kernel_regularizer = tf.keras.regularizers.l2(1e-3))(dropout)

# pooled_output = dropout(bert_model, training=False)
output_bert = Dense(
    units=train_labels.shape[1],
    activation="sigmoid",  # Choose a sigmoid for multi-label classification
    name='output_bert'
)(x)

In [ ]:
model = Model(inputs=inputs, outputs=output_bert, name='BERT_MultiLabel')

# for layer in model.layers[:3]:
#     layer.trainable = False

model.summary()

Model: "BERT_MultiLabel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 39)]              0         
                                                                 
 bert (TFBertMainLayer)      TFBaseModelOutputWithPoo  184345344 
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             39, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                   

### TRAIN MODEL

In [ ]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 8.5 MB/s 


In [ ]:
from tensorflow_addons.metrics import F1Score
from tensorflow_addons.optimizers import LazyAdam, Lookahead, MultiOptimizer
from tensorflow_addons.losses import SigmoidFocalCrossEntropy

In [ ]:
# fl = SigmoidFocalCrossEntropy(reduction = tf.keras.losses.Reduction.AUTO)
# loss = fl(
#     y_true = [[1.0, 0, 0 , 1.0], [1.0, 0, 0, 0]],y_pred = [[0.97, 0, 0, 0], [0, 1, 0, 1] ])
# loss

<tf.Tensor: shape=(), dtype=float32, numpy=15.356171>

In [ ]:
loss = BinaryCrossentropy(reduction = tf.keras.losses.Reduction.AUTO)

optimizers = [
    Lookahead(LazyAdam(1e-5), sync_period = 5),
    Adam(1e-5)
]
optimizers_and_layers = [(optimizers[0], model.layers[:2]), 
                         (optimizers[1], model.layers[2:])]
optimizer = MultiOptimizer(optimizers_and_layers)

metrics = [
    F1Score(num_classes = train_labels.shape[1],
            average = 'micro',
            threshold = 0.4),
           
    "binary_accuracy",
    
    AUC(name="average_precision", 
        curve="PR", 
        multi_label=True)
]
 
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

training_history = model.fit(
    train_dataset.shuffle(1000).batch(16).prefetch(tf.data.AUTOTUNE), 
    epochs=100, 
    validation_data=test_dataset.batch(16).prefetch(tf.data.AUTOTUNE)
)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['token_type_ids', 'attention_mask'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


62/62 [==============================] - 357s 5s/step - loss: 0.5830 - f1_score: 0.0725 - binary_accuracy: 0.8906 - average_precision: 0.0451 - val_loss: 0.4518 - val_f1_score: 0.0532 - val_binary_accuracy: 0.9536 - val_average_precision: 0.0901
Epoch 2/100
62/62 [==============================] - 298s 5s/step - loss: 0.4388 - f1_score: 0.0539 - binary_accuracy: 0.9579 - average_precision: 0.0456 - val_loss: 0.4077 - val_f1_score: 0.0123 - val_binary_accuracy: 0.9553 - val_average_precision: 0.0983
Epoch 3/100
62/62 [==============================] - 292s 5s/step - loss: 0.4031 - f1_score: 0.0421 - binary_accuracy: 0.9595 - average_precision: 0.0483 - val_loss: 0.3925 - val_f1_score: 0.0000e+00 - val_binary_accuracy: 0.9556 - val_average_precision: 0.0974
Epoch 4/100
62/62 [==============================] - 296s 5s/step - loss: 0.3876 - f1_score: 0.0149 - binary_accuracy: 0.9596 - average_precision: 0.0439 - val_loss: 0.3848 - val_f1_score: 0.0000e+00 - val_binary_accuracy: 0.9561 - va

In [ ]:
benchmarks = model.evaluate(
    test_dataset.batch(16), 
    return_dict=True
)

7/7 [==============================] - 1s 72ms/step - loss: 0.2590 - f1_score: 0.4689 - binary_accuracy: 0.9607 - average_precision: 0.2828


In [ ]:
def convert(label_string, label_onehot):
    labels = []
    for i, label in enumerate(label_string):
        if label_onehot[i]:
            labels.append(label)
    if len(labels) == 0:
        labels.append("NEUTRAL")
    return labels

In [ ]:
predictions = model.predict(test_dataset.batch(batch_size=1).take(10))

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['token_type_ids', 'attention_mask'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


10 sample prediction
predicted: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.] ['negative - FOOD#PRICES'] Actual Label: (['neutral - FOOD#PRICES', 'neutral - FOOD#STYLE_OPTIONS'])
predicted: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.] ['negative - RESTAURANT#GENERAL'] Actual Label: (['negative - RESTAURANT#GENERAL'])
predicted: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0.] ['positive - FOOD#QUALITY'] Actual Label: (['negative - FOOD#QUALITY'])
predicted: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.] ['negative - SERVICE#GENERAL'] Actual Label: (['positive - DRINKS#STYLE_OPTIONS', 'positive - SERVICE#GENERAL'])
predicted: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.] ['negative - SERVICE#GENERAL', 'positive - AMBIENCE#GENERAL

In [ ]:
preds=[]
labels = []
threshold = 0.4
for pred,(a,b) in zip(predictions, test_dataset):
    pred[pred>threshold]=1
    pred[pred<=threshold]=0
    name = str(convert(label_names, pred))
    labels.append(name)
    y.append(pred)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Creating plot
fig = plt.figure(figsize =(10, 7))
plt.pie(labels=labels)
 
# show plot
plt.show()

TypeError: ignored

<Figure size 720x504 with 0 Axes>

In [ ]:
predictions[:15]

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
loaded_model.summary()

Model: "BERT_MultiLabel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids (InputLayer)      [(None, 39)]              0         
                                                                 
 bert (TFBertMainLayer)      TFBaseModelOutputWithPoo  184345344 
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             39, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                   

In [ ]:
loaded_model.save(f"fine_tuned_bert")

INFO:tensorflow:Assets written to: fine_tuned_bert/assets


INFO:tensorflow:Assets written to: fine_tuned_bert/assets


In [ ]:
#import shutil
#shutil.make_archive('BERT_MODEL', 'zip', './')